# PWG2. Play Atari in the gym

> 本文是基于华盛顿大学 [Jeff Heaton](https://sites.wustl.edu/jeffheaton/)教授的 课程 [Applications of Deep Neural Networks](https://sites.wustl.edu/jeffheaton/t81-558/)。课程视频位于 [Youtube | Course on the Application of Deep Neural Networks ](https://www.youtube.com/watch?v=sRy26qWejOI&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&ab_channel=JeffHeaton)。代码仓库位于 [Github | t81_558_deep_learning](https://github.com/jeffheaton/t81_558_deep_learning)。本文的 Colab 链接位于：[Colab | AI_GYM.ipynb](https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb#scrollTo=dnID4yguIeX7)

check this [Github](https://github.com/openai/atari-py/issues/83)

In [ ]:
import gym
from gym.wrappers import Monitor
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()